In [39]:
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from IPython.display import clear_output

In [40]:
# Input path ke chromedriver.exe
path_chromedriver = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")

chrome_options = Options()
service = Service(executable_path=path_chromedriver)

In [41]:
# Open Chrome Browser
driver = webdriver.Chrome(service=service, options=chrome_options)
url = "https://www.google.com/maps"
driver.get(url)

In [42]:
driver.get(url)
time.sleep(3)
keyword = "Tempat Makan"
driver.find_elements(By.ID, "searchboxinput")[0].send_keys(keyword)
driver.find_elements(By.ID, "searchbox-searchbutton")[0].click()
time.sleep(10)

In [50]:
def scroll_gmaps():
    scroll_bar = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'

    scrollable_div = driver.find_element(By.XPATH,scroll_bar)
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)

    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

    while True:
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(10)  # Apabila jaringan low, bisa dinaikkan angkanya
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height

In [44]:
def inititate_data():
    maps_data = {
        'Name':[],
        'Type':[],
        'Rating':[],
        'Total Reviews':[],
        'Alamat':[],
        'Latitude':[],
        'Longitude':[],
        'No Hp':[],
        'Website':[],
        'Plus Code':[],
        'Last Reviewer':[],
        'Last Review Date':[],
        'Last Review Text':[]
    }
    return maps_data

In [67]:
def scrapping_data():
    time.sleep(5)
    name = driver.find_element(By.CLASS_NAME,"DUwDvf").text

    tipe = driver.find_element(By.CLASS_NAME,"DkEaL").text
    try: #contoh case "Warung Makan Barokah"
        rating = driver.find_element(By.CSS_SELECTOR,'div.F7nice span[aria-hidden="true"]').text
        rating = float(rating.replace(",", "."))
        total_reviews = driver.find_element(By.CSS_SELECTOR,'div.F7nice').text
    
    except:
        rating = "No Rating"
        total_reviews = "No Reviews"
    Alamat = driver.find_element(By.CLASS_NAME,"Io6YTe").text

    # Latitude dan Longitude dari Link Saat ini
    import re
    current_url = driver.current_url
    match = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", current_url)
    if match:
        latitude = match.group(1)
        longitude = match.group(2)

    try:
        no_hp = driver.find_element(By.XPATH, "//button[@data-tooltip='Salin nomor telepon']")
        no_hp = no_hp.get_attribute("aria-label")
    except:
        no_hp = "No Phone Number"
    no_hp

    try:
        website_element = driver.find_element(By.CSS_SELECTOR, 'a.CsEnBe')
        web = website_element.get_attribute('href')
    except:
        web = "No Website"
    web

    try:
        plus_code = driver.find_element(By.XPATH, "//button[@data-tooltip='Salin Plus Codes']")
        plus_code = plus_code.get_attribute("aria-label")
    except:
        plus_code = "No Plus Code"
    plus_code


    # Cek apakah ada review atau tidak, jika ada review maka tarik datanya
    try:
        try:
            driver.find_element(By.CLASS_NAME,'w8nwRe').click()
        except:
            skip
        
        last_reviewer = driver.find_element(By.CLASS_NAME,'d4r55').text
        last_review_date = driver.find_element(By.CLASS_NAME,'rsqaWe').text
        last_review_text = driver.find_element(By.CLASS_NAME,'wiI7pd').text
    except:
        last_reviewer = "No Review"
        last_review_date = "No Review"
        last_review_text = "No Review"

    keys = ['Name', 'Type', 'Rating', 'Total Reviews', 'Alamat', 'Latitude', 'Longitude', 'No Hp', 'Website', 'Plus Code', 'Last Reviewer', 'Last Review Date', 'Last Review Text']
    values = [name, tipe, rating, total_reviews, Alamat, latitude, longitude, no_hp, web, plus_code, last_reviewer, last_review_date, last_review_text]

    results = zip(keys, values)
    return results

In [1]:
# Kode awal, namun ada masalah beberapa lokasi di skip oleh coding
# def loop_scrapping_data(maps_data,locations):
#     index = 0
#     for data in locations:
#         try:
#             print(f"Processing index {index}")
#             data.click()
#             time.sleep(2)
#             hasil_scrapping = scrapping_data()    
#             for key,value in hasil_scrapping:
#                 maps_data[key].append(value)
#             df=pd.DataFrame(maps_data)
#             clear_output(wait=True)
#             display(df)
#             index+=1
#             print(f"Done processing index {index}")
#         except:
#             print(f"Error processing index {index}")

In [2]:
# Function baru dengan dynamic xpath, memastikan seluruh query akan diakses
def loop_scrapping_data(maps_data,locations):
    index = 3
    
    for i in range(0, len(locations)):
        try:
            print(f"Processing index {i}")
            base_xpath = f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{index}]/div/a'
            driver.find_element(By.XPATH, base_xpath).click()
            time.sleep(2) 
            
            hasil_scrapping = scrapping_data()
            
            for key, value in hasil_scrapping:
                maps_data[key].append(value)
            
            df = pd.DataFrame(maps_data)
            clear_output(wait=True)
            display(df)
            
            index += 2
            print(f"Done processing index {i}")
        except Exception as e:
            print(f"Error processing index {i}: {e}")

In [48]:
def count_query():
    locations = driver.find_elements(By.CLASS_NAME,"Nv2PK")
    Total_Query = len(locations)
    check_stats = driver.find_elements(By.CLASS_NAME,"DUwDvf")
    return locations,Total_Query, check_stats

In [ ]:
locations, Total_Query, check_stats = count_query()
maps_data = inititate_data()

if Total_Query>1:
    print("Update: Ditemukan multiple query, melakukan scrolling data")
    scroll_gmaps()
    locations, Total_Query, check_stats = count_query()
    print(f"Update: Total Query ditemukan untuk {keyword} sejumlah {Total_Query} lokasi, melakukan scrapping data")
    time.sleep(5)
    loop_scrapping_data(maps_data,locations)
else:
    scroll_gmaps()
    print(f'Ditemukan single query untuk {keyword}, melakukan scrapping data')
    data = scrapping_data()
    for key,value in data:
        maps_data[key].append(value)

,Name,Type,Rating,Total Reviews,Alamat,Latitude,Longitude,No Hp,Website,Plus Code,Last Reviewer,Last Review Date,Last Review Text
0,Pawon Resto Rantau,Restoran,4.1,"4,1\n(264)","By Pass, kelurahan, Jl. Rangda Malingkung, Ran...",-2.9360972,115.0229964,Telepon: 0856-4703-0000,No Website,"Plus Codes: 3578+HV Rangda Malingkung, Kabupat...",Nordiansyah,6 bulan lalu,"Pertama kali makan di sini, pas buka google ma..."
1,ADY JAYA RASA,Restoran,4.5,"4,5\n(57)","Jl. Datu Nuraya No.01, Rantau Kiwa, Kec. Tapin...",-2.9360972,115.0229964,No Phone Number,No Website,"Plus Codes: 3589+R8 Rantau Kiwa, Kabupaten Tap...",Amalia Alia,2 tahun lalu,"Tempat cukup strategis, di daerah rantau baru...."
2,Depot Aypa,Restoran Nasi Goreng,3.8,"3,8\n(59)","Unnamed Road, 71114, Ayunan Papan, Kec. Lokpai...",-2.9360972,115.0229964,Telepon: 0813-5129-3929,https://business.google.com/create?fp=59150589...,"Plus Codes: 352R+V4 Ayunan Papan, Kabupaten Ta...",Herman Civic,setahun lalu,Tempat nya sangat strategis hrga murah penjual...
3,Befo Cafe,Restoran,4.5,"4,5\n(40)","Jl. Jenderal Sudirman, RT.10/RW.2, Rantau Kiwa...",-2.9360972,115.0229964,Telepon: 0821-5536-9485,No Website,"Plus Codes: 3589+9G Rantau Kiwa, Kabupaten Tap...",Novia Ciptaningrum,6 bulan lalu,"Makanan lumayan enak, tapi sempet pesan beef b..."
4,Warung Lesehan Sido Mampir,Restoran,4.5,"4,5\n(46)","35C8+X9C, Rantau Kiwa, Kec. Tapin Utara, Kabup...",-2.9360972,115.0229964,No Phone Number,https://business.google.com/create?fp=95824169...,No Plus Code,Amalia Alia,4 tahun lalu,"Lokasi strategis di pinggir jalan raya, parkir..."
5,Warung Mbak Misti,Kedai Sarapan & Makan Siang,5.0,"5,0\n(2)","25FQ+W9, Shabah, Kec. Bungur, Kabupaten Tapin,...",-2.9360972,115.0229964,No Phone Number,https://business.google.com/create?fp=16370530...,No Plus Code,Fitria Djafar,2 bulan lalu,Nasi pecel lengkap dengan aneka gorengan langg...
6,Three'Fa Kafe,Restoran,4.3,"4,3\n(148)","Rangda Malingkung, Kec. Tapin Utara, Kabupaten...",-2.9360972,115.0229964,Telepon: 0822-5567-6876,https://wa.me/+6282255676876,"Plus Codes: 3569+M4 Rangda Malingkung, Kabupat...",Amalia Alia,3 tahun lalu,"Lokasinya memang agak masuk kedalam, tapi teta..."
7,Rumah Makan Sambal Cobek 3 Saudara,Kedai Sarapan & Makan Siang,4.6,"4,6\n(9)","Rangda Malingkung, Kec. Tapin Utara, Kabupaten...",-2.9360972,115.0229964,No Phone Number,No Website,"Plus Codes: 3568+86 Rangda Malingkung, Kabupat...",Official RatuBiogen,setahun lalu,Dari segi rasa middle alias engga istimewa.\n\...
8,KULINER ARBYSMOM,Restoran,5.0,"5,0\n(2)","Unnamed Road, Rangda Malingkung, Kec. Tapin Ut...",-2.9360972,115.0229964,Telepon: 0823-5412-2626,No Website,"Plus Codes: 353C+H7 Rangda Malingkung, Kabupat...",No Review,No Review,No Review
9,Plaza Kuliner Rantau Baru,Restoran,4.8,"4,8\n(22)","3598+X34, Jl. Datu Suban, Rantau Kiwa, Kec. Ta...",-2.9360972,115.0229964,No Phone Number,https://business.google.com/create?fp=51923059...,No Plus Code,Husnul Alfisyah Rina,5 bulan lalu,"Karena kebetulan ini deket sama kantor, jadi c..."


Done processing index 18
Processing index 18
Error processing index 18
Processing index 18


In [26]:
#maps_data = inititate_data()
maps_data

{'Name': [],
 'Type': [],
 'Rating': [],
 'Total Reviews': [],
 'Alamat': [],
 'Latitude': [],
 'Longitude': [],
 'No Hp': [],
 'Website': [],
 'Plus Code': [],
 'Last Reviewer': [],
 'Last Review Date': [],
 'Last Review Text': []}

In [ ]:
# Cleaning Data
